<div style="text-align: center; font-size: 38px;"><u>Option Pricing Project of <i>Quentin BORNE</i></u></div>

### **<u>READ_ME</u>:**
##### Dear Sir,<br><br>Please find below the code for my option pricing project, which is structured into three parts: the libraries used, the different classes (pricers and tabs), and finally the code for the user interface.<br><br>**The user interface includes two tabs :**<br><br>**- Equity Options Pricer:** This section allows pricing of European call and put options on equities using the Black-Scholes-Merton model, as well as both European and American call and put options using the Cox-Ross-Rubinstein model. It also calculates and displays the Greeks.</br><br>**- FX Options pricer:** This section allows pricing of European call and put options on FX using the Black-Scholes model with Monte Carlo simulation, as well as the pricing of knock-in options and enhanced collar strategies. It also includes functionality for plotting payoff graphs.<br><br>I personalized this project to reflect the activities of my current trading desk, where we frequently trade options for our FX hedging purposes (on EUR/USD, EUR/GBP, and EUR/JPY), focusing particularly on zero-cost strategies by trading enhanced collars (where the premiums net out). An enhanced collar consists of either a long call and a short put or a short call and a long put, with one of the two options having a knock-in barrier. When requesting prices from counterparties, we then ask for the barrier level of the option structure.<br><br>Regarding the sources that helped me, I can mention https://www.python.org/, https://github.com/ and https://chat.openai.com/. I would like to clarify that I used ChatGPT as a tool and did not merely copy and paste the code. I made consistent efforts to understand the workings of the code and adapted it as much as possible to my own needs.<br><br>Finally, I would like to thank you especially for your teaching and your availability, which we all greatly appreciated. I learned a lot thanks to you and really enjoyed the few days we spent together.<br><br>Best Regards,<br><br>Quentin BORNE


### **<u>LIBRARIES</u>:**

In [1]:
# Importing required libraries
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import ttk
from tkinter.ttk import Notebook
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

### **<u>CLASSES</u>:**

In [2]:
# CLASS OF THE BLACK-SCHOLES-MERTON EQUITY OPTIONS PRICER
class BSM_EQUITY_OPTIONS_PRICER():
    """
    This class allows pricing of equity options using the Black-Scholes-Merton model, as well as calculating Greeks.
    Parameters : 
                S: spot price of the underlying asset
                K: strike price
                r: annualized risk-free rate (as a percentage)
                q: annualized dividend yield (as a percentage)
                option_type: type of the option ('Call' or 'Put')
                sigma: annualized volatility of the underlying asset (as a percentage)
                pricing_date_str: pricing date in 'yyyy-mm-dd' format, as string data type
                expiration_date_str: expiration date in 'yyyy-mm-dd' format, as string data type
                T: time to expiration (in years)     
    """
    # CONSTRUCTOR
    def __init__(self, *args):
        if len(args) == 1 and isinstance(args[0], dict):  # using a dictionary as the first positional argument
            params = args[0]
            self.option_type = params.get('option_type')
            self.S = params.get('S')
            self.K = params.get('K')
            self.r = params.get('r') / 100 # convert percentage to decimal
            self.q = params.get('q') / 100
            self.sigma = params.get('sigma') / 100
            pricing_date_str = params.get('pricing_date_str')
            expiration_date_str = params.get('expiration_date_str')

        self.pricing_date = datetime.strptime(pricing_date_str, '%Y-%m-%d') # convert date from string data type to datetime object
        self.expiration_date = datetime.strptime(expiration_date_str, '%Y-%m-%d')
        self.days = (self.expiration_date - self.pricing_date).days # calculate number of days until expiration
        self.T = self.days / 365 # calculate T (time to expiration in year(s))

        self.d1 = (np.log(self.S / self.K) + ((self.r - self.q) + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T)) # compute d1 with parameters
        self.d2 = self.d1 - self.sigma * np.sqrt(self.T) # compute d2 as well


    # METHODS
    def European_Call_Price(self) -> float:
        # calculate price with d1, d2 and parameters using cdf() function from scipy.stats
        price = self.S * np.exp(-self.q * self.T) * norm.cdf(self.d1, 0, 1) - self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2, 0, 1) 
        return price

    def European_Put_Price(self) -> float:
        price = self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2, 0, 1) - self.S * np.exp(-self.q * self.T) * norm.cdf(-self.d1, 0, 1)
        return price

    def calculate_delta(self) -> float:
        if self.option_type == 'Call':
            delta = np.exp(-self.q * self.T) * norm.cdf(self.d1, 0, 1)
        elif self.option_type == 'Put':
            delta = np.exp(-self.q * self.T) * (norm.cdf(self.d1, 0, 1) - 1)
        return delta

    def calculate_gamma(self) -> float:
        gamma = np.exp(-self.q * self.T) * norm.pdf(self.d1, 0, 1) / (self.S * self.sigma * np.sqrt(self.T))
        return gamma

    def calculate_theta(self) -> float:
        if self.option_type == 'Call':
            theta = (-self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1, 0, 1) * self.sigma / (2 * np.sqrt(self.T))) - (self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2, 0, 1)) + (self.q * self.S * np.exp(-self.q * self.T) * norm.cdf(self.d1, 0, 1))
        elif self.option_type == 'Put':
            theta = (-self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1, 0, 1) * self.sigma / (2 * np.sqrt(self.T))) + (self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2, 0, 1)) - (self.q * self.S * np.exp(-self.q * self.T) * norm.cdf(-self.d1, 0, 1))
        return theta / 365  # convert theta from annual to daily rate of change

    def calculate_vega(self) -> float:
        vega = self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1, 0, 1) * np.sqrt(self.T)
        return vega / 100  # convert vega to represent the price change per one percentage point increase in volatility

    def calculate_rho(self) -> float:
        if self.option_type == 'Call':
            rho = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(self.d2, 0, 1)
        elif self.option_type == 'Put':
            rho = -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-self.d2, 0, 1)
        return rho / 100  # convert rho to represent the price change per one percentage point increase in interest rates


In [3]:
# CLASS OF THE COX-ROSS-RUBINSTEIN EQUITY OPTIONS PRICER
class CRR_EQUITY_OPTIONS_PRICER:
    """
    This class calculates prices for both European and American Call and Put options on equities using the Cox-Ross-Rubinstein model.
    Parameters : 
            S: spot price of the underlying asset
            K: strike price
            r: annualized risk-free rate (as a percentage)
            q: annualized dividend yield (as a percentage)
            sigma: annualized volatility of the underlying asset (as a percentage)
            pricing_date_str: pricing date in 'yyyy-mm-dd' format, as string data type
            expiration_date_str: expiration date in 'yyyy-mm-dd' format, as string data type
            T: time to expiration (in years)
            N: number of steps per year
            dt: time increment per period
            u: upward movement factor per period
            d: downward movement factor per period
            p: risk-neutral probability per period
    """
    # CONSTRUCTOR
    def __init__(self, *args):
        if len(args) == 1 and isinstance(args[0], dict):  # using a dictionary as the first positional argument
            params = args[0]
            self.S = params.get('S')
            self.K = params.get('K')
            self.r = params.get('r') / 100 # convert percentage to decimal
            self.q = params.get('q') / 100
            self.sigma = params.get('sigma') / 100
            pricing_date_str = params.get('pricing_date_str')
            expiration_date_str = params.get('expiration_date_str')

        self.pricing_date = datetime.strptime(pricing_date_str, '%Y-%m-%d') # convert date from string data type to datetime object
        self.expiration_date = datetime.strptime(expiration_date_str, '%Y-%m-%d')
        self.days = (self.expiration_date - self.pricing_date).days # calculate number of days until expiration
        self.T = self.days / 365 # calculate T (time to expiration in year(s))

        # Binomial Tree parameters
        self.N = int(252) # number of steps per year (I have choosen 252 since its the number of business days - might be a good compromise between pricing precision and pricing execution time)
        self.dt = 1 / self.N # time increment per period
        self.u = np.exp(self.sigma * np.sqrt(self.dt)) # upward movement factor per period
        self.d = np.exp(-self.sigma * np.sqrt(self.dt)) # downward movement factor per period
        self.p = (np.exp((self.r - self.q) * self.dt) - self.d) / (self.u - self.d) # risk-neutral probability per period
        
        # Constructing the stock price tree
        self.S_BinomialTree = np.zeros([int(self.N * self.T) + 1, int(self.N * self.T) + 1])
        for i in range(int(self.N * self.T) + 1):
            for j in range(i + 1):
                self.S_BinomialTree[int(self.N * self.T) - i + j, i] = self.S * (self.u ** (i - j)) * (self.d ** j)
        
        # Constructing the call price tree
        self.Call_binomialTree = np.zeros_like(self.S_BinomialTree)
        self.Call_binomialTree[:, int(self.N * self.T)] = np.maximum(self.S_BinomialTree[:, int(self.N * self.T)] - self.K, 0)

        # Constructing the put price tree
        self.Put_binomialTree = np.zeros_like(self.S_BinomialTree)
        self.Put_binomialTree[:, int(self.N * self.T)] = np.maximum(self.K - self.S_BinomialTree[:, int(self.N * self.T)], 0)
    
    
    # METHODS
    def European_Call_Price(self) -> float:
        # Traverses the tree backwards to find the present value of the option
        for i in range(int(self.N * self.T) - 1, -1, -1): # for each time period i (starting with the previous last until t=0)...
            for j in range(i + 1):            # ...iteration of j (vertical prices),...
                # ...to get each possible price of the tree
                self.Call_binomialTree[int(self.N * self.T) - i + j, i] = (self.p * self.Call_binomialTree[int(self.N * self.T) - i + j - 1, i + 1]
                                                            + (1 - self.p) * self.Call_binomialTree[int(self.N * self.T) - i + j, i + 1]) / (1 + (self.r - self.q) * self.dt)
        # The option price is at the bottom left of the tree
        European_Call_Price = self.Call_binomialTree[int(self.N * self.T), 0]
        return European_Call_Price

    def European_Put_Price(self) -> float: # Same logic but for the Put
        for i in range(int(self.N * self.T) - 1, -1, -1):
            for j in range(i + 1):
                self.Put_binomialTree[int(self.N * self.T) - i + j, i] = (self.p * self.Put_binomialTree[int(self.N * self.T) - i + j - 1, i + 1]
                                                            + (1 - self.p) * self.Put_binomialTree[int(self.N * self.T) - i + j, i + 1]) / (1 + (self.r - self.q) * self.dt)
        European_Put_Price = self.Put_binomialTree[int(self.N * self.T), 0]
        return European_Put_Price
    
    def American_Call_Price(self) -> float: # Same logic but for American Call
        for i in range(int(self.N * self.T) - 1, -1, -1):
            for j in range(i + 1):
                hold = (self.p * self.Call_binomialTree[int(self.N * self.T) - i + j - 1, i + 1] + 
                        (1 - self.p) * self.Call_binomialTree[int(self.N * self.T) - i + j, i + 1]) / (1 + (self.r - self.q) * self.dt)
                exercise = self.S_BinomialTree[int(self.N * self.T) - i + j, i] - self.K
                self.Call_binomialTree[int(self.N * self.T) - i + j, i] = max(hold, exercise) # American Options specificity : at each node, the value to keep is the highest between hold and exercise
        American_Call_Price = self.Call_binomialTree[int(self.N * self.T), 0]
        return American_Call_Price

    def American_Put_Price(self) -> float: # Same logic but for American Put
        for i in range(int(self.N * self.T) - 1, -1, -1):
            for j in range(i + 1):
                hold = (self.p * self.Put_binomialTree[int(self.N * self.T) - i + j - 1, i + 1] + 
                        (1 - self.p) * self.Put_binomialTree[int(self.N * self.T) - i + j, i + 1]) / (1 + (self.r - self.q) * self.dt)
                exercise = self.K - self.S_BinomialTree[int(self.N * self.T) - i + j, i]
                self.Put_binomialTree[int(self.N * self.T) - i + j, i] = max(hold, exercise)
        American_Put_Price = self.Put_binomialTree[int(self.N * self.T), 0]
        return American_Put_Price

In [4]:
# CLASS OF THE MONTE CARLO FX OPTIONS PRICER
class MC_FX_OPTIONS_PRICER:
    """
    This class allows modeling and pricing of FX options using the Black-Scholes model with Monte Carlo simulation. 
    It supports standard European Call and Put, knock-in options, and barrier levels for a zero-cost strategy while trading an enhanced collar.
    Parameters:
                S: spot FX rate
                K: strike FX rate
                N: number of simulation
                T: time to expiration (in years)
                rb: risk-free rate of the base currency (annualized)
                rq: risk-free rate of the quote currency (annualized)
                sigma: volatility of the currency pair (annualized)
                notional: notional of the option (in quote currency)
                pricing_date_str: pricing date in 'yyyy-mm-dd' format, as string data type
                expiration_date_str: expiration date in 'yyyy-mm-dd' format, as string data type
                option_type: type of the option ('Call' or 'Put')
                barrier: barrier level of the knock-in option
    """
    # CONSTRUCTOR
    def __init__(self, *args):
        if len(args) > 0 and isinstance(args[0], dict):  # Using a dictionary as the first positional argument
            params = args[0]
            self.S = params.get('S')
            self.K = params.get('K')
            self.rb = params.get('rb') / 100
            self.rq = params.get('rq') / 100
            self.sigma = params.get('sigma') / 100
            self.notional = params.get('notional')
            pricing_date_str = params.get('pricing_date_str')
            expiration_date_str = params.get('expiration_date_str')
            self.N = params.get('N')
            self.option_type = params.get('option_type')
            self.barrier = params.get('barrier', None)
            if len(args) == 2: # if there is 2 args, the second is the target_price
                self.target_price = args[1] 

        # Common initialization...
        # ...about data management
        self.pricing_date = datetime.strptime(pricing_date_str, '%Y-%m-%d') # convert date from string data type to datetime object
        self.expiration_date = datetime.strptime(expiration_date_str, '%Y-%m-%d')
        self.T = (self.expiration_date - self.pricing_date).days / 365 # calculate T (time to expiration in year(s))

        # ...about Monte Carlo method
        np.random.seed(1)  # for reproducibility
        self.Z = np.random.standard_normal(self.N)
        self.S_T = self.S * np.exp((self.rq - self.rb - 0.5 * self.sigma**2) * self.T + self.sigma * np.sqrt(self.T) * self.Z)


    # METHODS
    def European_Call_Price(self) -> float:
        payoff = np.maximum(self.S_T - self.K, 0) * self.notional # compute payoff
        price = np.mean(payoff) * np.exp(-self.rq * self.T) # calculate price
        return price

    def European_Put_Price(self) -> float:
        payoff = np.maximum(self.K - self.S_T, 0) * self.notional
        price = np.mean(payoff) * np.exp(-self.rq * self.T)
        return price

    def European_Up_and_In_Call_Price(self) -> float:
        hit = self.S_T >= self.barrier # where the barrier is activated
        payoff = np.where(hit, np.maximum(self.S_T - self.K, 0), 0) * self.notional # payoff is then calculated where the barrier has been activated
        price = np.mean(payoff) * np.exp(-self.rq * self.T)
        return price
    
    def European_Down_and_In_Put_Price(self) -> float:
        hit = self.S_T <= self.barrier
        payoff = np.where(hit, np.maximum(self.K - self.S_T, 0), 0) * self.notional
        price = np.mean(payoff) * np.exp(-self.rq * self.T)
        return price
    
    def ZeroCost_EuropeanKI_barrier_on_Call(self) -> float:
        """
        This function calculates the barrier (European knock-in barrier) level to have on a Call option in order to get a zero-cost strategy while trading an enhanced collar.
        """
        price_tolerance = 0.00001
        max_iterations = 100
        lower_barrier = min(self.S_T)  # set initial minimum for barrier
        upper_barrier = max(self.S_T)  # set initial maximum for barrier
        estimated_barrier = (lower_barrier + upper_barrier) / 2  # initial guess for barrier is the midpoint of the range

        for _ in range(max_iterations): # Iteration until we get a satisfying value of the barrier
            hit = self.S_T >= estimated_barrier
            payoff = np.where(hit, np.maximum(self.S_T - self.K, 0), 0) * self.notional
            estimated_price = np.mean(payoff) * np.exp(-self.rq * self.T)
            if abs(estimated_price - self.target_price) <= price_tolerance: # check if the estimated price is close enough to the target price
                return estimated_barrier
            if estimated_price > self.target_price: # adjust the barrier range based on whether the estimated price was too high or too low
                lower_barrier = estimated_barrier
            else:
                upper_barrier = estimated_barrier
            estimated_barrier = (lower_barrier + upper_barrier) / 2 # update the estimated barrier for the next iteration

        return estimated_barrier # Return the best estimate after max_iterations
    
    def ZeroCost_EuropeanKI_barrier_on_Put(self) -> float:
        """
        This function does the same thing than the previous one but for Put options.
        """
        price_tolerance = 0.00001
        max_iterations = 100
        lower_barrier = min(self.S_T) 
        upper_barrier = max(self.S_T)
        estimated_barrier = (lower_barrier + upper_barrier) / 2 

        for _ in range(max_iterations):
            hit = self.S_T <= estimated_barrier
            payoff = np.where(hit, np.maximum(self.K - self.S_T, 0), 0) * self.notional
            estimated_price = np.mean(payoff) * np.exp(-self.rq * self.T)
            if abs(estimated_price - self.target_price) <= price_tolerance:
                return estimated_barrier
            if estimated_price > self.target_price:
                upper_barrier = estimated_barrier
            else:
                lower_barrier = estimated_barrier
            estimated_barrier = (lower_barrier + upper_barrier) / 2

        return estimated_barrier  

In [5]:
# CLASS OF THE EQUITY OPTIONS PRICER TAB
class TAB_EQUITY_OPTIONS_PRICER():

    # CONSTRUCTOR
    def __init__(self):
        self.setup_ui() # Initialize UI elements and store them in attributes
        self.fetch_parameters() # Fetch parameters and store them in attributes

    # METHODS
    def setup_ui(self):
        """
        This function sets up User Interface elements for this options pricing tab including layout, events bindings and default values.
        """
        # Setup Labels with consistent font, size and spacing
        font_ui = "Consolas"
        size = 10
        self.label_title_parameters = ttk.Label(tabEquity, text="Parameters", font=("font_ui", 14, "underline"), anchor="w")
        self.label_pricing = ttk.Label(tabEquity,              text="Pricing model:  . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_option_type = ttk.Label(tabEquity,          text="Option type:  . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_exercise_style = ttk.Label(tabEquity,       text="Exercise style: . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_S = ttk.Label(tabEquity,                    text="Spot price: . . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_K = ttk.Label(tabEquity,                    text="Strike: . . . . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_r = ttk.Label(tabEquity,                    text="Annualized risk-free interest rate: . . .", font=(font_ui, size))
        self.label_q = ttk.Label(tabEquity,                    text="Annualized dividend yield:  . . . . . . .", font=(font_ui, size))
        self.label_sigma = ttk.Label(tabEquity,                text="Annualized volatility:  . . . . . . . . .", font=(font_ui, size))
        self.label_pricing_date = ttk.Label(tabEquity,         text="Pricing date (yyyy-mm-dd):  . . . . . . .", font=(font_ui, size))
        self.label_expiration_date = ttk.Label(tabEquity,      text="Expiration date (yyyy-mm-dd): . . . . . .", font=(font_ui, size))
        self.label_title_results = ttk.Label(tabEquity, text="Results", font=("font_ui", 14, "underline"), anchor="w")
        self.label_price = ttk.Label(tabEquity, text="")
        self.label_days = ttk.Label(tabEquity, text="")
        self.label_delta = ttk.Label(tabEquity, text="")
        self.label_gamma = ttk.Label(tabEquity, text="")
        self.label_theta = ttk.Label(tabEquity, text="")
        self.label_vega = ttk.Label(tabEquity, text="")
        self.label_rho = ttk.Label(tabEquity, text="")
        
        # Setup Entry fields and Comboboxes
        self.combo_pricing = ttk.Combobox(tabEquity, values=["Black-Scholes-Merton", "Cox-Ross-Rubinstein"], width=24)
        self.combo_option_type = ttk.Combobox(tabEquity, values=["Call", "Put"])
        self.combo_exercise_style = ttk.Combobox(tabEquity, values=["European"])
        self.entry_S = ttk.Entry(tabEquity)
        self.entry_K = ttk.Entry(tabEquity)
        self.entry_r = ttk.Entry(tabEquity)
        self.entry_q = ttk.Entry(tabEquity)
        self.entry_sigma = ttk.Entry(tabEquity)
        self.entry_pricing_date = ttk.Entry(tabEquity)
        self.entry_expiration_date = ttk.Entry(tabEquity)
    
        # Setup Bouttons
        self.button_calculate_results = ttk.Button(tabEquity, text="Calculate Results", command=self.on_calculate_results)

        # Positioning Widgets in the grid layout...
        #...of the first title...
        self.label_title_parameters.grid(row=0, column=0, columnspan=2, sticky="w")
         #...then other widgets by creating a list...
        widgets = [
            self.label_pricing, self.combo_pricing, 
            self.label_option_type, self.combo_option_type, 
            self.label_exercise_style, self.combo_exercise_style, 
            self.label_S, self.entry_S, 
            self.label_K, self.entry_K, 
            self.label_r, self.entry_r,
            self.label_q, self.entry_q, 
            self.label_sigma, self.entry_sigma, 
            self.label_pricing_date, self.entry_pricing_date, 
            self.label_expiration_date, self.entry_expiration_date
        ]
        #...then arranging widgets in rows and columns; apply padding for spacing
        for i, widget in enumerate(widgets):
            widget.grid(row=(i // 2) + 1, column=i % 2, sticky='ew', padx=3, pady=1)
        #...then of the second title...
        self.button_calculate_results.grid(row=11, column=0, sticky='ew', padx=3, pady=1)
        #...finally of last widgets (results ones)
        self.label_title_results.grid(row=12, column=0, columnspan=2, sticky="w")
        self.label_price.grid(row=13, column=0, columnspan=2)
        self.label_days.grid(row=14, column=0, columnspan=2)
        self.label_delta.grid(row=15, column=0, columnspan=2)
        self.label_gamma.grid(row=16, column=0, columnspan=2)
        self.label_theta.grid(row=17, column=0, columnspan=2)
        self.label_vega.grid(row=18, column=0, columnspan=2)
        self.label_rho.grid(row=19, column=0, columnspan=2)

        # Function to append a '%' sign to required Entry fields on focus loss, if not already present
        def append_percent(event, entry_widget):
            current_value = entry_widget.get() # retrieve input from the Entry field desired
            if not current_value.endswith('%'):
                entry_widget.delete(0, tk.END)
                entry_widget.insert(0, current_value + '%')
        
        # Function to append a '$' sign to required Entry fields on focus loss, if not already present
        def append_dollar(event, entry_widget):
            current_value = entry_widget.get()
            if not current_value.startswith('$'):
                entry_widget.delete(0, tk.END)
                entry_widget.insert(0, '$' + current_value)

        # Function to update the display of the options of the exercise style based on the pricing model
        def update_exercise_style_options(event):
            pricing_model = self.combo_pricing.get()
            if pricing_model == "Cox-Ross-Rubinstein":
                self.combo_exercise_style['values'] = ["European", "American"]
                self.combo_exercise_style.current(0)  # optionally set the first value by default
            else:
                self.combo_exercise_style['values'] = ["European"]
                self.combo_exercise_style.current(0)

        # Bind pricing model change to update display of exercise style options
        self.combo_pricing.bind('<<ComboboxSelected>>', update_exercise_style_options)
    
        # Bind <FocusOut> event to append_percent for the relevant Entry fields
        self.entry_r.bind("<FocusOut>", lambda event: append_percent(event, self.entry_r))
        self.entry_q.bind("<FocusOut>", lambda event: append_percent(event, self.entry_q))
        self.entry_sigma.bind("<FocusOut>", lambda event: append_percent(event, self.entry_sigma))

        # Bind <FocusOut> event to append_percent for the relevant Entry fields
        self.entry_S.bind("<FocusOut>", lambda event: append_dollar(event, self.entry_S))
        self.entry_K.bind("<FocusOut>", lambda event: append_dollar(event, self.entry_K))

        # Initialize default values, dropdown selections and barrier display
        self.combo_pricing.current(0) # set the first value of the Combobox by default
        self.combo_option_type.current(0)
        self.combo_exercise_style.current(0)
        self.entry_S.insert(0, "$100.00")
        self.entry_K.insert(0, "$100.00")
        self.entry_r.insert(0, "5%")
        self.entry_q.insert(0, "0%")
        self.entry_sigma.insert(0, "20%")
        self.entry_pricing_date.insert(0, datetime.today().strftime('%Y-%m-%d')) # date of today by default
        self.entry_expiration_date.insert(0, (datetime.today() + timedelta(days=365)).strftime('%Y-%m-%d')) # date of today + 365 days by default

    def fetch_parameters(self):
        """
        This function retrieves parameters from the widgets and stores them in attributes for further processing.
        """
        # Attempt to parse the dates to ensure they are in the correct format
        try:
            pricing_date = datetime.strptime(self.entry_pricing_date.get(), '%Y-%m-%d')
            expiration_date = datetime.strptime(self.entry_expiration_date.get(), '%Y-%m-%d')
        except ValueError:  # if there is a date format error, raise a custom ValueError
            raise ValueError("Dates must be in 'yyyy-mm-dd' format.")

        # Collecting and formatting input values (params) in a dictionnary
        self.params = {
            'pricing_model': str(self.combo_pricing.get()),
            'option_type': str(self.combo_option_type.get()),
            'exercise_style': str(self.combo_exercise_style.get()),
            'S': float(self.entry_S.get().replace('$', '').strip()), # removing '$' from spot input, converting to float, and stripping white spaces
            'K': float(self.entry_K.get().replace('$', '').strip()),
            'r': float(self.entry_r.get().replace('%', '').strip()),
            'q': float(self.entry_q.get().replace('%', '').strip()), 
            'sigma': float(self.entry_sigma.get().replace('%', '').strip()),
            'pricing_date_str': str(self.entry_pricing_date.get()),
            'expiration_date_str': str(self.entry_expiration_date.get()),
        }
    
    def on_calculate_results(self):
        """
        This function handles the calculations process for the desired option and displays the result.
        """
        # Re-setup the labels
        self.label_price.config(text="")
        self.label_days.config(text="")
        self.label_delta.config(text="")
        self.label_gamma.config(text="")
        self.label_theta.config(text="")
        self.label_vega.config(text="")
        self.label_rho.config(text="")
        
        # Retrieve user-inputted parameters from the UI widgets
        try:
            self.fetch_parameters()
        except ValueError as e: # if ValueError
            self.label_price.config(text=str(e))  # show the error message from the exception
            return  # stop the rest of the function from executing
        
        # Check if any parameter is negative, or not as it should be, and add specific error messages if necessary
        error_messages = [] # initialize an error message list
        if self.params['S'] < 0: 
            error_messages.append("Spot price cannot be negative.")
        if self.params['K'] < 0:
            error_messages.append("Strike price cannot be negative.")
        if self.params['q'] < 0:
            error_messages.append("Dividend yield cannot be negative.")
        if self.params['sigma'] < 0:
            error_messages.append("Volatility cannot be negative.")
        if BSM_EQUITY_OPTIONS_PRICER(self.params).T <= 0:
            error_messages.append("Expiration date must be at least one day later than the pricing date.")

        # If there are any error messages, update the UI and return
        if error_messages:
            errorMessage = "Error: " + "; ".join(error_messages)
            self.label_price.config(text=errorMessage)
            # Clear other results since calculation was not performed
            self.label_days.config(text="")
            self.label_delta.config(text="")
            self.label_gamma.config(text="")
            self.label_theta.config(text="")
            self.label_vega.config(text="")
            self.label_rho.config(text="")
            return  # exit the function to prevent further calculations

        # Create instances of the pricing models with fetched parameters
        BSM_pricer = BSM_EQUITY_OPTIONS_PRICER(self.params)
        BINOMIAL_pricer = CRR_EQUITY_OPTIONS_PRICER(self.params)

        # Continue with the option pricing process according to the pricing model and the option type
        if self.params['pricing_model'] == 'Black-Scholes-Merton':
            if self.params['option_type'] == 'Call':
                price = BSM_pricer.European_Call_Price()
            elif self.params['option_type'] == 'Put':
                price = BSM_pricer.European_Put_Price()
        elif self.params['pricing_model'] == 'Cox-Ross-Rubinstein':
            if self.params['exercise_style'] == 'European':
                if self.params['option_type'] == 'Call':
                    price = BINOMIAL_pricer.European_Call_Price()
                elif self.params['option_type'] == 'Put':
                    price = BINOMIAL_pricer.European_Put_Price()
            elif self.params['exercise_style'] == 'American':
                if self.params['option_type'] == 'Call':
                    price = BINOMIAL_pricer.American_Call_Price()
                elif self.params['option_type'] == 'Put':
                    price = BINOMIAL_pricer.American_Put_Price()

        # Perform the calculations of days and greeks
        days = BSM_pricer.days
        delta = BSM_pricer.calculate_delta()
        gamma = BSM_pricer.calculate_gamma()
        theta = BSM_pricer.calculate_theta()
        vega = BSM_pricer.calculate_vega()
        rho = BSM_pricer.calculate_rho()

        # Update the UI with all the results
        exercise_style = self.params['exercise_style']
        option_type = self.params['option_type']
        self.label_price.config(text=f"{exercise_style} {option_type} Option Price: ${price:.2f}")
        self.label_days.config(text=f"Days until expiration: {days}")
        self.label_delta.config(text=f"Delta: {delta:.4f}")
        self.label_gamma.config(text=f"Gamma: {gamma:.4f}")
        self.label_theta.config(text=f"Theta: {theta:.4f}")
        self.label_vega.config(text=f"Vega: {vega:.4f}")
        self.label_rho.config(text=f"Rho: {rho:.4f}")

In [6]:
# CLASS OF THE FX OPTIONS PRICER TAB
class TAB_FX_OPTIONS_PRICER():

    # CONSTRUCTOR
    def __init__(self):
        self.setup_ui() # Initialize UI elements and store them in attributes
        self.fetch_parameters() # Fetch parameters and store them in attributes

    # METHODS
    def setup_ui(self):
        """
        This function sets up User Interface elements for this options pricing tab including layout, events bindings and default values.
        """
        # Setup Labels with consistent font, size and spacing...
        #...for both options
        font_ui = "Consolas"
        size = 10
        self.label_currency = ttk.Label(tabFX,        text="Currency pair: . . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_S = ttk.Label(tabFX,               text="Spot FX rate:  . . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_K = ttk.Label(tabFX,               text="Strike:  . . . . . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_rb = ttk.Label(tabFX,              text="Annualized risk-free rate (base ccy):  . . .", font=(font_ui, size))
        self.label_rq = ttk.Label(tabFX,              text="Annualized risk-free rate (quote ccy): . . .", font=(font_ui, size))
        self.label_sigma = ttk.Label(tabFX,           text="Annualized volatility: . . . . . . . . . . .", font=(font_ui, size))
        self.label_notional = ttk.Label(tabFX,        text="Notional (quote ccy):  . . . . . . . . . . .", font=(font_ui, size))
        self.label_pricing_date = ttk.Label(tabFX,    text="Pricing date (yyyy-mm-dd): . . . . . . . . .", font=(font_ui, size))
        self.label_expiration_date = ttk.Label(tabFX, text="Expiration date (yyyy-mm-dd):  . . . . . . .", font=(font_ui, size))
        self.label_N = ttk.Label(tabFX,               text="Number of simulations: . . . . . . . . . . .", font=(font_ui, size))        
        self.label_position = ttk.Label(tabFX,        text="Position:  . . . . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_option_type = ttk.Label(tabFX,     text="Option type: . . . . . . . . . . . . . . . .", font=(font_ui, size))
        self.label_barrier = ttk.Label(tabFX,         text="Barrier level: . . . . . . . . . . . . . . .", font=(font_ui, size), state='hidden')
        #...for the first option
        self.label_price = ttk.Label(tabFX, text="Price:", font=(font_ui, size))
        self.label_ZC_barrier = ttk.Label(tabFX, text="", font=(font_ui, size))
        #...for the second option
        self.label_price_n2 = ttk.Label(tabFX, text="Price:", font=(font_ui, size))
        self.label_ZC_barrier_n2 = ttk.Label(tabFX, text="", font=(font_ui, size))
        
        # Setup Entry fields and Comboboxes...
        #...for the first option
        self.combo_currency = ttk.Combobox(tabFX, values=['EUR/USD', 'EUR/GBP', 'EUR/JPY'], width=24)
        self.entry_S = ttk.Entry(tabFX)
        self.entry_K = ttk.Entry(tabFX)
        self.entry_rb = ttk.Entry(tabFX)
        self.entry_rq = ttk.Entry(tabFX)
        self.entry_sigma = ttk.Entry(tabFX)
        self.entry_notional = ttk.Entry(tabFX)
        self.entry_pricing_date = ttk.Entry(tabFX)
        self.entry_expiration_date = ttk.Entry(tabFX)
        self.entry_N = ttk.Entry(tabFX)
        self.combo_position = ttk.Combobox(tabFX, values=['Long', 'Short'])
        self.combo_option_type = ttk.Combobox(tabFX, values=['European Call', 'European Put', 'European Up-and-In Call', 'European Down-and-In Put'])
        self.entry_barrier = ttk.Entry(tabFX, state='hidden')
        #...for the second option
        self.combo_currency_n2 = ttk.Combobox(tabFX, values=['EUR/USD', 'EUR/GBP', 'EUR/JPY'], width=24)
        self.entry_S_n2 = ttk.Entry(tabFX)
        self.entry_K_n2 = ttk.Entry(tabFX)
        self.entry_rb_n2 = ttk.Entry(tabFX)
        self.entry_rq_n2 = ttk.Entry(tabFX)
        self.entry_sigma_n2 = ttk.Entry(tabFX)
        self.entry_notional_n2 = ttk.Entry(tabFX)
        self.entry_pricing_date_n2 = ttk.Entry(tabFX)
        self.entry_expiration_date_n2 = ttk.Entry(tabFX)
        self.entry_N_n2 = ttk.Entry(tabFX)
        self.combo_position_n2 = ttk.Combobox(tabFX, values=['Long', 'Short'])
        self.combo_option_type_n2 = ttk.Combobox(tabFX, values=['European Call', 'European Put', 'European Up-and-In Call', 'European Down-and-In Put'])
        self.entry_barrier_n2 = ttk.Entry(tabFX, state='hidden')

        # Setup Bouttons
        self.button_calculate_prices = ttk.Button(tabFX, text="Calculate Options Prices", command=self.on_calculate_prices)
        self.button_calculate_ZC_barrier = ttk.Button(tabFX, text="Calculate Barrier for Zero-Cost Strategy", command=self.on_calculate_ZC_barrier, state='disabled')  # initially disabled
        self.button_draw_payout = ttk.Button(tabFX, text="Show Options Payoff", command=self.draw_options_payoff)

        # Positioning Widgets in the grid layout...
        #...by creating a list of tuples...
        widgets = [
            (self.label_currency, self.combo_currency, self.combo_currency_n2),
            (self.label_S, self.entry_S, self.entry_S_n2),
            (self.label_K, self.entry_K, self.entry_K_n2),
            (self.label_rb, self.entry_rb, self.entry_rb_n2),
            (self.label_rq, self.entry_rq, self.entry_rq_n2),
            (self.label_sigma, self.entry_sigma, self.entry_sigma_n2),
            (self.label_notional, self.entry_notional, self.entry_notional_n2),
            (self.label_pricing_date, self.entry_pricing_date, self.entry_pricing_date_n2),
            (self.label_expiration_date, self.entry_expiration_date, self.entry_expiration_date_n2),
            (self.label_N, self.entry_N, self.entry_N_n2),
            (self.label_position, self.combo_position, self.combo_position_n2),
            (self.label_option_type, self.combo_option_type, self.combo_option_type_n2),
            (self.label_barrier, self.entry_barrier, self.entry_barrier_n2),
            (self.button_calculate_prices, self.label_price, self.label_price_n2),
            (self.button_calculate_ZC_barrier, self.label_ZC_barrier, self.label_ZC_barrier_n2),
            (self.button_draw_payout, None, None)
        ]
        #...then arranging widgets in rows with labels in the first column and corresponding entries/comboboxes in the second and third columns
        for row_index, widget_group in enumerate(widgets):
            for col_index, widget in enumerate(widget_group):
                if widget:  # check if the widget exists to avoid trying to place NoneType in the grid
                    widget.grid(row=row_index, column=col_index, sticky='ew', padx=3, pady=1)

        # Function to update the display of the barrier based on the selected option types for both options, showing or hiding the barrier level input as needed
        def update_barrier_display(event):
            self.fetch_parameters()  # Retrieve user-inputted parameters from the UI widgets
            # Check if either option has a barrier
            option1_has_barrier = self.params['option1']['option_type'] in ['European Up-and-In Call', 'European Down-and-In Put']
            option2_has_barrier = self.params['option2']['option_type'] in ['European Up-and-In Call', 'European Down-and-In Put']
            # Update the display for both options
            if option1_has_barrier or option2_has_barrier:
                self.label_barrier.grid()  # always show the label if any option has a barrier
                # Adjust the visibility of the barrier entries for each option
                if option1_has_barrier:
                    self.entry_barrier.grid()
                else:
                    self.entry_barrier.grid_remove()
                if option2_has_barrier:
                    self.entry_barrier_n2.grid()
                else:
                    self.entry_barrier_n2.grid_remove()
            else: # If neither option has a barrier, remove both the label and entry displays
                self.label_barrier.grid_remove()
                self.entry_barrier.grid_remove()
                self.entry_barrier_n2.grid_remove()
            # Update the display of the Calculate Barrier for Zero-Cost Strategy button
            if option1_has_barrier != option2_has_barrier:  # only one option has a barrier
                self.button_calculate_ZC_barrier['state'] = 'normal'  # enable the button
            else:
                self.button_calculate_ZC_barrier['state'] = 'disabled'  # disable the button

        # Function to append a '%' sign to required Entry fields on focus loss, if not already present
        def append_percent(event, entry_widget):
            current_value = entry_widget.get() # retrieve input from the Entry field desired
            if not current_value.endswith('%'):
                entry_widget.delete(0, tk.END)
                entry_widget.insert(0, current_value + '%')

        # Function to handle FocusOut event with two functions for some Entry fields
        def handle_focus_out(event, entry_widget):
            append_percent(event, entry_widget)
            self.update_linked_fields()

        # Bind option type selection change to update the barrier field display
        self.combo_option_type.bind('<<ComboboxSelected>>', update_barrier_display) # for the first option
        self.combo_option_type_n2.bind('<<ComboboxSelected>>', update_barrier_display) # for the second option
        
        # Bind ComboboxSelected and FocusOut events to update_linked_fields function for each necessary first option's entry fields
        self.combo_currency.bind("<<ComboboxSelected>>", lambda event: self.update_linked_fields())
        self.entry_S.bind("<FocusOut>", lambda event: self.update_linked_fields())
        self.entry_notional.bind("<FocusOut>", lambda event: self.update_linked_fields())
        self.entry_pricing_date.bind("<FocusOut>", lambda event: self.update_linked_fields())
        self.entry_expiration_date.bind("<FocusOut>", lambda event: self.update_linked_fields())
        self.entry_N.bind("<FocusOut>", lambda event: self.update_linked_fields())

        # Bind the FocusOut event to handle_focus_out function for some specific Entry fields
        self.entry_rb.bind("<FocusOut>", lambda event, entry=self.entry_rb: handle_focus_out(event, entry))
        self.entry_rq.bind("<FocusOut>", lambda event, entry=self.entry_rq: handle_focus_out(event, entry))
        self.entry_sigma.bind("<FocusOut>", lambda event, entry=self.entry_sigma: handle_focus_out(event, entry))

        # Initialize default values, dropdown selections and barrier display...
        #...for the first option
        self.combo_currency.current(0) # set the first value of the Combobox by default
        self.entry_S.insert(0, "1.1000")
        self.entry_K.insert(0, "1.0500")
        self.entry_rb.insert(0, "5%")
        self.entry_rq.insert(0, "5%")
        self.entry_sigma.insert(0, "10%")
        self.entry_notional.insert(0, "100.00")
        self.entry_pricing_date.insert(0, datetime.today().strftime('%Y-%m-%d')) # date of today by default
        self.entry_expiration_date.insert(0, (datetime.today() + timedelta(days=365)).strftime('%Y-%m-%d'))
        self.entry_N.insert(0, "1000000")
        self.combo_position.current(0)
        self.combo_option_type.current(1)
        #...for the second option
        self.entry_K_n2.insert(0, "1.1000")
        self.combo_position_n2.current(1)
        self.combo_option_type_n2.current(2)
        self.entry_barrier_n2.insert(0, "1.2500")
        self.update_linked_fields() # call the function for updating other Entry fields
        update_barrier_display(None) # do not display the barrier by default
    
    def update_linked_fields(self):
        """
        This function updates Entry fields of the second option to match the first option where required.
        """
        # Set the state of the linked fields to modifiable (i.e normal state)
        self.combo_currency_n2.configure(state='normal')
        self.entry_S_n2.configure(state='normal')
        self.entry_rb_n2.configure(state='normal')
        self.entry_rq_n2.configure(state='normal')
        self.entry_sigma_n2.configure(state='normal')
        self.entry_notional_n2.configure(state='normal')
        self.entry_pricing_date_n2.configure(state='normal')
        self.entry_expiration_date_n2.configure(state='normal')
        self.entry_N_n2.configure(state='normal')

        # Copy values from the first option to the second
        self.combo_currency_n2.set(self.combo_currency.get())
        self.entry_S_n2.delete(0, tk.END)
        self.entry_S_n2.insert(0, self.entry_S.get())
        self.entry_rb_n2.delete(0, tk.END)
        self.entry_rb_n2.insert(0, self.entry_rb.get())
        self.entry_rq_n2.delete(0, tk.END)
        self.entry_rq_n2.insert(0, self.entry_rq.get())
        self.entry_sigma_n2.delete(0, tk.END)
        self.entry_sigma_n2.insert(0, self.entry_sigma.get())
        self.entry_notional_n2.delete(0, tk.END)
        self.entry_notional_n2.insert(0, self.entry_notional.get())
        self.entry_pricing_date_n2.delete(0, tk.END)
        self.entry_pricing_date_n2.insert(0, self.entry_pricing_date.get())
        self.entry_expiration_date_n2.delete(0, tk.END)
        self.entry_expiration_date_n2.insert(0, self.entry_expiration_date.get())
        self.entry_N_n2.delete(0, tk.END)
        self.entry_N_n2.insert(0, self.entry_N.get())

        # Disable the linked fields to prevent editing
        self.combo_currency_n2.configure(state='disabled')
        self.entry_S_n2.configure(state='disabled')
        self.entry_rb_n2.configure(state='disabled')
        self.entry_rq_n2.configure(state='disabled')
        self.entry_sigma_n2.configure(state='disabled')
        self.entry_notional_n2.configure(state='disabled')
        self.entry_pricing_date_n2.configure(state='disabled')
        self.entry_expiration_date_n2.configure(state='disabled')
        self.entry_N_n2.configure(state='disabled')
    
    def fetch_parameters(self):
        """
        This function retrieves parameters for both options from the widgets and stores them in nested dictionaries for further processing.
        """
        try:
            # Attempt to parse the dates to ensure they are in the correct format
            pricing_date = datetime.strptime(self.entry_pricing_date.get(), '%Y-%m-%d')
            expiration_date = datetime.strptime(self.entry_expiration_date.get(), '%Y-%m-%d')
        except ValueError:
            # If there is a date format error, raise a custom ValueError
            raise ValueError("Dates must be in 'yyyy-mm-dd' format.")

        # Update Entry fields of the second option to match the first option
        self.update_linked_fields()

        # Collecting and formatting input values for the first and second options in nested dictionaries
        self.params = {
            'option1': {
                'currency_pair': str(self.combo_currency.get()),
                'S': float(self.entry_S.get()),
                'K': float(self.entry_K.get()),
                'rb': float(self.entry_rb.get().replace('%', '').strip()),  # remove '%' from rate inputs, convert to float, and strip white spaces
                'rq': float(self.entry_rq.get().replace('%', '').strip()),
                'sigma': float(self.entry_sigma.get().replace('%', '').strip()),
                'notional': float(self.entry_notional.get()),
                'pricing_date_str': str(self.entry_pricing_date.get()),
                'expiration_date_str': str(self.entry_expiration_date.get()),
                'N': int(self.entry_N.get()),
                'position': str(self.combo_position.get()),
                'option_type': str(self.combo_option_type.get()),
                # convert barrier text to float if specified, otherwise set to None
                'barrier': float(self.entry_barrier.get()) if self.entry_barrier.get() != '' and self.combo_option_type.get() in ['European Up-and-In Call', 'European Down-and-In Put'] else None
            },
            'option2': { # process similarly as for option1
                'currency_pair': str(self.combo_currency_n2.get()),
                'S': float(self.entry_S_n2.get()),
                'K': float(self.entry_K_n2.get()),
                'rb': float(self.entry_rb_n2.get().replace('%', '').strip()),  
                'rq': float(self.entry_rq_n2.get().replace('%', '').strip()),
                'sigma': float(self.entry_sigma_n2.get().replace('%', '').strip()),
                'notional': float(self.entry_notional_n2.get()),
                'pricing_date_str': str(self.entry_pricing_date_n2.get()),
                'expiration_date_str': str(self.entry_expiration_date_n2.get()),
                'N': int(self.entry_N_n2.get()),
                'position': str(self.combo_position_n2.get()),
                'option_type': str(self.combo_option_type_n2.get()),
                'barrier': float(self.entry_barrier_n2.get()) if self.entry_barrier_n2.get() != '' and self.combo_option_type_n2.get() in ['European Up-and-In Call', 'European Down-and-In Put'] else None
            }
        }

    def on_calculate_prices(self):
        """
        This function handles the calculation process for two options and displays the results.
        """
        # If there is an error in fetch_parameters(), then update the display of the UI (with an appropriate error message)
        try:
            self.fetch_parameters()
        except ValueError as e: # if ValueError
            self.label_price.config(text=str(e))  # set the error message
            self.label_price.grid_forget() # forget previous grid placement
            self.label_price.grid(row=13, column=1, columnspan=2, sticky='ew', padx=3, pady=1) # span of two columns instead of one only
            self.label_price_n2.grid_remove() # ensure the second price label is not displayed
            return  # stop the rest of the function from executing

        # Check if any parameter is negative or not as it should be and add specific error messages if necessary
        error_messages = [] # initialize an error message list
        if self.params['option1']['S'] < 0:
            error_messages.append("Spot cannot be negative.")
        if self.params['option1']['K'] < 0 or self.params['option2']['K'] < 0:
            error_messages.append("Strike cannot be negative.")
        if self.params['option1']['notional'] < 0:
            error_messages.append("Notional cannot be negative.")
        if self.params['option1']['sigma'] < 0:
            error_messages.append("Volatility cannot be negative.")
        if self.params['option1']['N'] < 100000:
            error_messages.append("Number of simulations cannot be lower than 100000.")
        if MC_FX_OPTIONS_PRICER(self.params['option1']).T <= 0:
            error_messages.append("Expiration date must be at least one day later than the pricing date.")

        # If there are any error messages, update the UI and return
        if error_messages:
            errorMessage = "Error: " + "; ".join(error_messages)
            self.label_price.config(text=errorMessage)
            self.label_price.grid_forget()
            self.label_price.grid(row=13, column=1, columnspan=2, sticky='ew', padx=3, pady=1)
            self.label_price_n2.grid_remove()
            return  # Exit the function to prevent further calculations

        # Re-setup the grid of the label_price_n2 if ever it has been removed with a previous error (above code)
        self.label_price_n2.grid(row=13, column=2, sticky='ew', padx=3, pady=1)

        # Create instances of the FX options pricer with fetched parameters for both options
        pricer1 = MC_FX_OPTIONS_PRICER(self.params['option1'])
        pricer2 = MC_FX_OPTIONS_PRICER(self.params['option2'])

        # Determine the price based on the type of each option selected by the user
        price_info = []  # to store the results for both options
        for pricer, option in zip([pricer1, pricer2], ['option1', 'option2']):
            if self.params[option]['option_type'] == 'European Call':
                price = pricer.European_Call_Price()
            elif self.params[option]['option_type'] == 'European Put':
                price = pricer.European_Put_Price()
            elif self.params[option]['option_type'] == 'European Up-and-In Call':
                price = pricer.European_Up_and_In_Call_Price()
            elif self.params[option]['option_type'] == 'European Down-and-In Put':
                price = pricer.European_Down_and_In_Put_Price()
            price_info.append(f"{price:.2f}")

        # Update the UI with the calculated option prices in the specified currency
        self.label_price.config(text=f"Price in {self.params['option1']['currency_pair'].split('/')[1]}: {price_info[0]}")
        self.label_price_n2.config(text=f"Price in {self.params['option1']['currency_pair'].split('/')[1]}: {price_info[1]}")

    def on_calculate_ZC_barrier(self):
        """
        This function calculates the barrier level that makes the two options a zero-cost strategy while trading an enhanced collar and then displays the result on the UI.
        """
        # If there is an error in fetch_parameters(), then update the display of the UI (with an appropriate error message)
        try:
            self.fetch_parameters()
        except ValueError as e: # if ValueError
            self.label_ZC_barrier.config(text=str(e))  # set the error message
            self.label_ZC_barrier.grid_forget() # forget previous grid placement
            self.label_ZC_barrier.grid(row=14, column=1, columnspan=2, sticky='ew', padx=3, pady=1) # span of two columns instead of one only
            self.label_ZC_barrier_n2.grid_remove() # ensure the second price label is not displayed
            return  # stop the rest of the function from executing
            
        # Check if any parameter is negative or not as it should be and add specific error messages if necessary
        error_messages = [] # initialize an error message list
        if self.params['option1']['S'] < 0:
            error_messages.append("Spot cannot be negative.")
        if self.params['option1']['K'] < 0 or self.params['option2']['K'] < 0:
            error_messages.append("Strike cannot be negative.")
        if self.params['option1']['notional'] < 0:
            error_messages.append("Notional cannot be negative.")
        if self.params['option1']['sigma'] < 0:
            error_messages.append("Volatility cannot be negative.")
        if self.params['option1']['N'] < 100000:
            error_messages.append("Number of simulations cannot be lower than 100000.")
        if MC_FX_OPTIONS_PRICER(self.params['option1']).T <= 0:
            error_messages.append("Expiration date must be at least one day later than the pricing date.")

        # If there are any error messages, update the UI and return
        if error_messages:
            errorMessage = "Error: " + "; ".join(error_messages)
            self.label_ZC_barrier.config(text=errorMessage)
            self.label_ZC_barrier.grid_forget()
            self.label_ZC_barrier.grid(row=14, column=1, columnspan=2, sticky='ew', padx=3, pady=1)
            self.label_ZC_barrier_n2.grid_remove()
            return  # Exit the function to prevent further calculations

        # Re-setup the grid of the label_price_n2 if ever it has been removed with an error above
        self.label_ZC_barrier_n2.grid(row=14, column=2, sticky='ew', padx=3, pady=1)

        # Create instances of the FX options pricer with fetched parameters for both options
        pricer1 = MC_FX_OPTIONS_PRICER(self.params['option1'])
        pricer2 = MC_FX_OPTIONS_PRICER(self.params['option2'])

        # Determine the price based on the type of each option selected by the user
        price_info = []  # to store the results for both options
        for pricer, option in zip([pricer1, pricer2], ['option1', 'option2']):
            if self.params[option]['option_type'] == 'European Call':
                price = pricer.European_Call_Price()
            elif self.params[option]['option_type'] == 'European Put':
                price = pricer.European_Put_Price()
            elif self.params[option]['option_type'] == 'European Up-and-In Call':
                price = pricer.European_Up_and_In_Call_Price()
            elif self.params[option]['option_type'] == 'European Down-and-In Put':
                price = pricer.European_Down_and_In_Put_Price()
            price_info.append(f"{price:.2f}")    
        option1_price = float(price_info[0]) # get price of option1
        option2_price = float(price_info[1]) # get price of option2

        # Determine the barrier for a zero-cost strategy according to parameters
        if self.params['option1']['barrier'] != None:
            has_barrier = 'option1' # will be useful later for displaying the result 
            target_price = option2_price # target price is the option's price without barrier
            if self.params['option1']['option_type'] == 'European Up-and-In Call':
                ZC_barrier = MC_FX_OPTIONS_PRICER(self.params['option1'], target_price).ZeroCost_EuropeanKI_barrier_on_Call()
            elif self.params['option1']['option_type'] == 'European Down-and-In Put':
                ZC_barrier = MC_FX_OPTIONS_PRICER(self.params['option1'], target_price).ZeroCost_EuropeanKI_barrier_on_Put()
        elif self.params['option2']['barrier'] != None:
            has_barrier = 'option2'
            target_price = option1_price
            if self.params['option2']['option_type'] == 'European Up-and-In Call':
                ZC_barrier = MC_FX_OPTIONS_PRICER(self.params['option2'], target_price).ZeroCost_EuropeanKI_barrier_on_Call()
            elif self.params['option2']['option_type'] == 'European Down-and-In Put':
                ZC_barrier = MC_FX_OPTIONS_PRICER(self.params['option2'], target_price).ZeroCost_EuropeanKI_barrier_on_Put()
        
        # Manage display of the result on the UI
        if has_barrier == 'option1':
            if self.params['option1']['option_type'] == 'European Up-and-In Call':
                if ZC_barrier < self.params['option1']['K']: # if the solution found for ZC_barrier is below K...
                    self.label_ZC_barrier.config(text=f"Error: ZC_Barrier < K") # ...display an error message since it is impossible (with an Up-and-In Call)
                    self.label_ZC_barrier_n2.config(text=f"")
                else:  
                    self.label_ZC_barrier.config(text=f"ZC Barrier: {ZC_barrier:.4f}") # else display the satisfying result of ZC_barrier  
                    self.label_ZC_barrier_n2.config(text=f"")
            if self.params['option1']['option_type'] == 'European Down-and-In Put':
                if ZC_barrier > self.params['option1']['K']:
                    self.label_ZC_barrier.config(text=f"Error: ZC_Barrier > K")
                    self.label_ZC_barrier_n2.config(text=f"")
                else:    
                    self.label_ZC_barrier.config(text=f"ZC Barrier: {ZC_barrier:.4f}")
                    self.label_ZC_barrier_n2.config(text=f"")
        elif has_barrier == 'option2':
            if self.params['option2']['option_type'] == 'European Up-and-In Call':
                if ZC_barrier < self.params['option2']['K']:
                    self.label_ZC_barrier_n2.config(text=f"Error: ZC_Barrier < K")
                    self.label_ZC_barrier.config(text=f"")
                else:    
                    self.label_ZC_barrier_n2.config(text=f"ZC Barrier: {ZC_barrier:.4f}")
                    self.label_ZC_barrier.config(text=f"")
            if self.params['option2']['option_type'] == 'European Down-and-In Put':
                if ZC_barrier > self.params['option2']['K']:
                    self.label_ZC_barrier_n2.config(text=f"Error: ZC_Barrier > K")
                    self.label_ZC_barrier.config(text=f"")
                else:    
                    self.label_ZC_barrier_n2.config(text=f"ZC Barrier: {ZC_barrier:.4f}")
                    self.label_ZC_barrier.config(text=f"")

    def draw_options_payoff(self):
        """
        This function draws the options payoff graph for the Enhanced Collar strategy based on parameters.
        """
        # Retrieve user-inputted parameters from the UI widgets
        self.fetch_parameters()

        # Define a range of stock prices for plotting the option payoff graph
        S_values = np.linspace(0.5 * self.params['option1']['S'], 1.5 * self.params['option1']['S'], 100000)

        # Initialize combined payoff array
        combined_payoff = np.zeros(len(S_values))

        # Calculate payoffs for both options based on their type and position
        for option in ['option1', 'option2']:
            payoff = np.zeros(len(S_values))
            position_multiplier = 1 if self.params[option]['position'] == 'Long' else -1
            if self.params[option]['option_type'] == 'European Call':
                payoff = np.maximum(S_values - self.params[option]['K'], 0)
            elif self.params[option]['option_type'] == 'European Put':
                payoff = np.maximum(self.params[option]['K'] - S_values, 0)
            elif self.params[option]['option_type'] == 'European Up-and-In Call':
                payoff = np.where(S_values > self.params[option]['barrier'], np.maximum(S_values - self.params[option]['K'], 0), 0)
            elif self.params[option]['option_type'] == 'European Down-and-In Put':
                payoff = np.where(S_values <= self.params[option]['barrier'], np.maximum(self.params[option]['K'] - S_values, 0), 0)
            # Adjust payoff for the position and accumulate to combined payoff
            combined_payoff += position_multiplier * payoff * self.params[option]['notional']

        # Create and display the option payoff graph using the calculated payoff values
        fig = Figure(figsize=(7.75, 3), dpi=75)
        plot = fig.add_subplot(111)
        plot.plot(S_values, combined_payoff)
        plot.set_title('ENHANCED COLLAR STRATEGY')
        plot.set_xlabel('Spot Price at Maturity')
        plot.set_ylabel(f"Payoff ({self.params['option1']['currency_pair'].split('/')[1]})")
        
        # Automatically adjust subplot parameters to give specified padding and avoid elements overlap
        fig.tight_layout(pad=2.0)

        # Integrate and display the matplotlib graph within the Tkinter window
        canvas = FigureCanvasTkAgg(fig, master=tabFX) 
        canvas_widget = canvas.get_tk_widget()
        canvas_widget.grid(row=28, column=0, columnspan=3, pady=5)
        canvas.draw()

### **<u>USER INTERFACE</u>:**

In [7]:
# CREATING THE MAIN WINDOW AND TABS
# Initialize the main Tkinter window
root = tk.Tk()

# Set the title of the main window
root.title("OPTIONS PRICER")

# Create a tab control widget (like a notebook with tabs)
tabControl = Notebook(root)

# Create two tabs as Tkinter Frames: 'tabEquity' and 'tabFX'
tabEquity = ttk.Frame(tabControl)
tabFX = ttk.Frame(tabControl)

# Add the two tabs to the tab control, naming them 'Equity Options Pricer' and 'FX Options Pricer (enhanced collar)' respectively
tabControl.add(tabEquity, text='Equity Options Pricer')
tabControl.add(tabFX, text='FX Options Pricer (enhanced collar)')

# Configure the tab control to fill the available space and expand if extra space is available
tabControl.pack(expand=1, fill="both")

# Instantiate classes for the first and the second tab
TAB_1 = TAB_EQUITY_OPTIONS_PRICER()
TAB_2 = TAB_FX_OPTIONS_PRICER()

# Start the Tkinter event loop to display the window and waiting for events (like button clicks)
root.mainloop()